In [22]:
import zipfile
import os

def unzip_folder(zip_path, extract_to):
    """
    Unzip a zip archive to a specified directory.

    Parameters:
        zip_path (str): Path to the zip archive.
        extract_to (str): Directory where the contents will be extracted.

    Returns:
        None
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

if __name__ == "__main__":
    # Specify the path to the zip archive and the directory to extract to
    zip_path = '/content/points.zip'
    extract_to = '/content'

    # Create the extract directory if it doesn't exist
    os.makedirs(extract_to, exist_ok=True)

    # Unzip the folder
    unzip_folder(zip_path, extract_to)

    print(f"Successfully extracted contents to: {extract_to}")

Successfully extracted contents to: /content


In [23]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [24]:
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Path to the dataset folder
# dataset_path = '/Users/dvirani/dp/dataset/'

# Load annotations (image filenames and coordinates) from CSV
annotations_file = pd.read_csv(f'/content/points/red_dot_coordinates.csv')

# Prepare lists to store images and corresponding coordinates
images = []
coordinates = []

# Loop through each row in the annotations file
for idx, row in annotations_file.iterrows():
    # Read and preprocess image
    image_path = f'/content/points/images/red_dot_{idx}.png'
    image = cv2.imread(image_path)  # Load image using OpenCV
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Extract coordinates of the red dot
    x, y = row['x'], row['y']

    # Store image and coordinates
    images.append(image)
    coordinates.append((x, y))

# Convert lists to NumPy arrays
images = np.array(images)
coordinates = np.array(coordinates)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, coordinates, test_size=0.2, random_state=42)

# Normalize pixel values (optional preprocessing)
X_train = X_train / 255.0  # Scale pixel values to [0, 1]
X_test = X_test / 255.0

# Display shapes of the datasets
print(f"Training images shape: {X_train.shape}")
print(f"Training coordinates shape: {y_train.shape}")
print(f"Test images shape: {X_test.shape}")
print(f"Test coordinates shape: {y_test.shape}")


Training images shape: (8000, 100, 100, 3)
Training coordinates shape: (8000, 2)
Test images shape: (2000, 100, 100, 3)
Test coordinates shape: (2000, 2)


In [25]:
max_train=y_train.max()
max_test=y_test.max()

In [26]:
# Normalize coordinates
y_train_norm = y_train / max_train  # Replace max_value with the appropriate normalization factor
y_val_norm = y_test / max_train


In [27]:
y_train=y_train_norm
y_test=y_val_norm

In [28]:
X_train[0].shape

(100, 100, 3)

In [29]:
input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])
input_shape

(100, 100, 3)

In [30]:
# input_shape = (X_train.shape[0], X_train.shape[2], X_train.shape[3])
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(32,activation='relu'),
        layers.Dense(8,activation='relu'),
        layers.Dense(2)  # Output layer for x and y coordinates (2 values)
    ])
    return model

model = create_model()
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 conv2d_7 (Conv2D)           (None, 19, 19, 128)      

In [31]:
# Compile the model with custom optimizer and learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='mse')
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# ###########    much better 0.33#########

In [32]:
import tensorflow as tf

# Assuming you have defined and compiled your model (`model`) properly
# X_train and y_train_norm are your training data

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
]

# Split your training data into training and validation sets
# Here, assuming you want to use 10% of the data for validation
validation_split = 0.1
split_index = int(len(X_train) * validation_split)
X_val = X_train[-split_index:]
y_val = y_train_norm[-split_index:]
X_train = X_train[:-split_index]
y_train = y_train_norm[:-split_index]

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=32, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)


Epoch 1/32
225/225 [==============================] - 5s 15ms/step - loss: 0.0475 - val_loss: 4.0119e-04
Epoch 2/32
  6/225 [..............................] - ETA: 2s - loss: 4.1040e-04

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


225/225 [==============================] - 3s 15ms/step - loss: 3.7451e-04 - val_loss: 3.0623e-04
Epoch 3/32
225/225 [==============================] - 3s 13ms/step - loss: 4.0869e-04 - val_loss: 3.1179e-04
Epoch 4/32
225/225 [==============================] - 3s 14ms/step - loss: 4.1425e-04 - val_loss: 2.7956e-04
Epoch 5/32
225/225 [==============================] - 3s 13ms/step - loss: 4.1247e-04 - val_loss: 3.5355e-04
Epoch 6/32
225/225 [==============================] - 3s 15ms/step - loss: 3.3786e-04 - val_loss: 2.7185e-04
Epoch 7/32
225/225 [==============================] - 3s 13ms/step - loss: 4.2841e-04 - val_loss: 3.8389e-04
Epoch 8/32
225/225 [==============================] - 3s 13ms/step - loss: 3.5652e-04 - val_loss: 2.8541e-04
Epoch 9/32
225/225 [==============================] - 3s 14ms/step - loss: 3.2195e-04 - val_loss: 1.7118e-04
Epoch 10/32
225/225 [==============================] - 3s 14ms/step - loss: 2.5899e-04 - val_loss: 2.3100e-04
Epoch 11/32
225/225 [========

In [33]:
import h5py
model.save_weights('point.h5')

In [34]:
import json
from keras.models import load_model
config = model.get_config()
print(config)
with open('model.json', 'w+') as f:
    json.dump(config, f)

{'name': 'sequential_1', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 100, 100, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'conv2d_4_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_4', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 100, 100, 3), 'filters': 32, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'registe

In [35]:
model.summary() #((3*3)*3)*32 + 32

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 conv2d_7 (Conv2D)           (None, 19, 19, 128)      

In [41]:
import numpy as np

# Assuming model is already defined and trained
model.load_weights('point.h5')

# Predict coordinates using the model
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = np.mean(np.abs(y_pred - y_test))

# Calculate baseline error based on a simple benchmark (e.g., mean or median of y_train)
# Replace y_train_mean with the appropriate benchmark value
y_train_mean = np.mean(y_train)  # Calculate mean of the training target coordinates

# Calculate baseline error as a percentage of the benchmark
baseline_error = (mae / np.mean(np.abs(y_test - y_train_mean))) * 100

print("Baseline Error (MAE): %.2f%%" % baseline_error)


63/63 [==============================] - 0s 5ms/step
Baseline Error (MAE): 1.98%


In [37]:
# # Adjust model architecture and hyperparameters
# model = models.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(256, activation='relu'),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(2)  # Output layer for x and y coordinates (2 values)
# ])

# # Compile the model with a lower learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimizer, loss='mse')

# # Train the model with data augmentation
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest'
# )

# history = model.fit(datagen.flow(X_train, y_train_norm, batch_size=32),
#                     epochs=50,
#                     validation_data=(X_test, y_test))

# # Evaluate the model
# loss = model.evaluate(X_test, y_test)
# print(f'Validation Loss: {loss}')


In [38]:
# Assuming X_train and y_train are your image data and corresponding coordinates
# model.fit(X_train, y_train, epochs=32, batch_size=16, validation_split=0.1)


In [39]:
# # Assuming X_val and y_val are your validation image data and corresponding coordinates
# loss = model.evaluate(X_test, y_test)
# print(f'Validation Loss: {loss}')
